In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stat
import pylab 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
pd.set_option('display.max_columns', None)

df = pd.read_csv("C:/Project/Electricity_project/RawData/GlobalEYUtility_Final_for Demo.csv")
df = df[:300]
df.shape

(300, 119)

In [3]:
df.head()

,abin,Feeder_Category,BREAKER_MAKE,UpcaseRoadTraffic,Upcase_DepthOfCable,Upcase_SoilCondition,Upcase_LeadExposed,Upcase_ArmourCableCondition,CABLE_SIZE,NO__OF_PARALLEL_RUNNING_CABLES,FaultCurrent_Count,_5Yr_Faults_Count,No_of_Load_transfer_Operations,No_of_Times_Exceeded100_,No_of_times_exceeded_60_,No_of_time_below_60_,No_of_Times_Exceeded90_,LENGTH_OF_CABLE_MTRS_,NO__OF_JOINTS,AGE_YRS_,Number_OF_Cables,PILC,XLPE,DATE_ONLY,DAY_OF_WEEK,WEEKEND_Y_N_,Average_Current,MORNING_AVG_CURRENT,AFTERNOON_AVG_CURRENT,EVENING_AVG_CURRENT,Peak_Current,Peak_Slab,Average_voltage,MORNING_AVG_VOLTAGE,AFTERNOON_AVG_VOLTAGE,EVENING_AVG_VOLTAGE,Peak_Voltage,target,Avg_Heat_Index,Morning_AVG_Heat_Index,Afternoon_AVG_Heat_Index,Evening_Avg_Heat_Index,Peak_HI,Avg_humidity,Morning_AVG_humidity,Afternoon_AVG_HUMIDITY,Evening_Avg_HUMIDITY,Max_Humidity,Avg_TEMP,Morning_AVG_TEMP,Afternoon_AVG_TEMP,Evening_Avg_TEMP,Max_TEMP,IR_Value_After_JointingPHASE_TO,MORNING_AVG_CURRENT_N_1,MORNING_AVG_CURRENT_N_2,AFTERNOON_AVG_CURRENT_N_1,AFTERNOON_AVG_CURRENT_N_2,EVENING_AVG_CURRENT_N_1,EVENING_AVG_CURRENT_N_2,MORNING_AVG_VOLTAGE_N_1,MORNING_AVG_VOLTAGE_N_2,AFTERNOON_AVG_VOLTAGE_N_1,AFTERNOON_AVG_VOLTAGE_N_2,EVENING_AVG_VOLTAGE_N_1,EVENING_AVG_VOLTAGE_N_2,TARGET_N_1,TARGET_N_2,AVERAGE_CURRENT_N_1,AVERAGE_CURRENT_N_2,AVERAGE_VOLTAGE_N_1,AVERAGE_VOLTAGE_N_2,AVERAGE_HEAT_INDEX_N_1,AVERAGE_HEAT_INDEX_N_2,AVERAGE_HUMUDITY_N_1,AVERAGE_HUMUDITY_N_2,AVERAGE_TEMP_N_1,AVERAGE_TEMP_N_2,MORNING_AVG_HEAT_INDEX_N_1,MORNING_AVG_HEAT_INDEX_N_2,AFTERNOON_AVG_HEAT_INDEX_N_1,AFTERNOON_AVG_HEAT_INDEX_N_2,EVENING_AVG_HEAT_INDEX_N_1,EVENING_AVG_HEAT_INDEX_N_2,MORNING_AVG_HUMUDITY_N_1,MORNING_AVG_HUMUDITY_N_2,AFTERNOON_AVG_HUMUDITY_N_1,AFTERNOON_AVG_HUMUDITY_N_2,EVENING_AVG_HUMUDITY_N_1,EVENING_AVG_HUMUDITY_N_2,MORNING_AVG_TEMP_N_1,MORNING_AVG_TEMP_N_2,AFTERNOON_AVG_TEMP_N_1,AFTERNOON_AVG_TEMP_N_2,EVENING_AVG_TEMP_N_1,EVENING_AVG_TEMP_N_2,Peak_Current_N_1,Peak_Current_N_2,Peak_Current_Time_N_1,Peak_Current_Time_N_2,Peak_Current_Slab_N_1,Peak_Current_Slab_N_2,New_PublicHoliday,New_Summer,New_Monsoon,New_Winter,_dataobs_,Feeder,Switch,Section Id,Pathid,Path Order,Lat,Long,Customer At Risk,Revenue At Rsik,Assets At Risk,Risk Type,NoFaultsSince2016
0,ED,RES,SIEMENS VCB,MODERATE,NORMAL,DRY,NO,STRONG,120.0,3.0,2.0,5.0,0.0,0.0,0.0,96.0,0.0,586.6,7.0,36.0,2.0,1.0,1.0,08-01-2016,6.0,N,4.352917,9.688485,1.55125,1.565161,43.99,M,11.047187,11.035455,11.026250,11.081290,11.20,1.0,26.667515,23.993545,28.256467,27.873791,31.224829,60.078646,70.946061,48.330312,60.637419,79.34,25.887917,22.996970,27.952188,26.834516,31.11,250.0,68.730303,19.471818,108.005000,20.005000,107.405161,21.148710,11.035455,10.961818,11.026250,11.001875,11.081290,10.954839,0.0,0.0,94.310625,20.191042,11.047187,10.972917,26.667515,26.667515,60.078646,60.078646,25.887917,25.887917,23.993545,23.993545,28.256467,28.256467,27.873791,27.873791,70.946061,70.946061,48.330312,48.330312,60.637419,60.637419,22.996970,22.996970,27.952188,27.952188,26.834516,26.834516,1.5,9.08,0.0,0.0,M,M,N,N,N,Y,379.0,30462.0,10499.0,30462-10499-1,SANTACRUZ RECEIVING STATION-ANAND NAGAR NO.2 S...,2.0,19.080542,72.845893,8000.0,97.0,10.0,Low,0.0
1,ED,RES,ABB SF6,HEAVY,NORMAL,MOIST,NO,STRONG,300.0,1.0,7.0,2.0,0.0,0.0,0.0,96.0,0.0,533.1,5.0,NaN,1.0,0.0,1.0,15-01-2016,6.0,N,4.416875,12.658182,0.10000,0.100000,19.24,M,11.021667,10.977576,11.050000,11.039355,11.27,1.0,24.975674,22.618742,26.488964,25.922561,28.202562,69.590937,74.704545,63.073125,70.875484,76.92,23.927083,21.308182,25.579687,25.009032,27.30,350.0,58.424242,34.783333,79.528125,53.851250,88.881613,52.395484,10.977576,10.977576,11.050000,11.050000,11.039355,11.039355,0.0,0.0,75.294062,46.826563,11.021667,11.021667,24.975674,24.975674,69.590937,69.590937,23.927083,23.927083,22.618742,22.618742,26.488964,26.488964,25.922561,25.922561,74.704545,74.704545,63.073125,63.073125,70.875484,70.875484,21.308182,21.308182,25.579687,25.579687,25.009032,25.009032,0.1,18.46,0.0,0.0,M,M,N,N,N,Y,705.0,30462.0,10499.0

In [6]:
def plot_data(df,feature):
    plt.figure(figsize=(10,6))
    plt.subplot(1,2,1)
    df[feature].hist()
    plt.subplot(1,2,2)
    stat.probplot(df[feature],dist='norm',plot=pylab)
    plt.show()

In [7]:
df1 = df[['Average_Current','target']]
df1

,Average_Current,target
0,4.352917,1.0
1,4.416875,1.0
2,14.331250,1.0
3,27.535312,1.0
4,0.845625,1.0
...,...,...
295,63.178125,1.0
296,60.295521,1.0
297,94.052188,0.0
298,71.319583,1.0


In [8]:
a = df1.pct_change()
a

,Average_Current,target
0,NaN,NaN
1,0.014693,0.0
2,2.244658,0.0
3,0.921348,0.0
4,-0.969289,0.0
...,...,...
295,-0.473396,inf
296,-0.045627,0.0
297,0.559854,-1.0
298,-0.241702,inf


In [10]:
pd.concat([df1,a],axis=1).head(30)

,Average_Current,target,Average_Current,target
0,4.352917,1.0,NaN,NaN
1,4.416875,1.0,0.014693,0.0
2,14.331250,1.0,2.244658,0.0
3,27.535312,1.0,0.921348,0.0
4,0.845625,1.0,-0.969289,0.0
5,9.230417,1.0,9.915496,0.0
6,0.000000,0.0,-1.000000,-1.0
7,58.210104,1.0,inf,inf
8,60.773229,0.0,0.044032,-1.0
9,27.929583,1.0,-0.540429,inf


In [11]:
df1 = df[['Peak_Current','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(30)

,Peak_Current,target,Peak_Current,target
0,43.99,1.0,NaN,NaN
1,19.24,1.0,-0.562628,0.0
2,20.80,1.0,0.081081,0.0
3,109.47,1.0,4.262981,0.0
4,80.23,1.0,-0.267105,0.0
5,25.88,1.0,-0.677427,0.0
6,0.00,0.0,-1.000000,-1.0
7,137.21,1.0,inf,inf
8,71.58,0.0,-0.478318,-1.0
9,98.03,1.0,0.369517,inf


In [12]:
df1 = df[['Average_voltage','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(30)

,Average_voltage,target,Average_voltage,target
0,11.047187,1.0,NaN,NaN
1,11.021667,1.0,-0.002310,0.0
2,11.066146,1.0,0.004036,0.0
3,11.221042,1.0,0.013997,0.0
4,11.077917,1.0,-0.012755,0.0
5,11.104688,1.0,0.002417,0.0
6,11.127500,0.0,0.002054,-1.0
7,11.161458,1.0,0.003052,inf
8,10.972604,0.0,-0.016920,-1.0
9,11.003750,1.0,0.002839,inf


In [13]:
df1 = df[['Peak_Voltage','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(30)

,Peak_Voltage,target,Peak_Voltage,target
0,11.20,1.0,NaN,NaN
1,11.27,1.0,0.006250,0.0
2,11.49,1.0,0.019521,0.0
3,11.41,1.0,-0.006963,0.0
4,11.39,1.0,-0.001753,0.0
5,11.37,1.0,-0.001756,0.0
6,11.34,0.0,-0.002639,-1.0
7,11.40,1.0,0.005291,inf
8,11.12,0.0,-0.024561,-1.0
9,11.31,1.0,0.017086,inf


In [14]:
df1 = df[['Avg_Heat_Index','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(30)

,Avg_Heat_Index,target,Avg_Heat_Index,target
0,26.667515,1.0,NaN,NaN
1,24.975674,1.0,-0.063442,0.0
2,25.394108,1.0,0.016754,0.0
3,25.914466,1.0,0.020491,0.0
4,26.553766,1.0,0.024670,0.0
5,26.158478,1.0,-0.014886,0.0
6,26.772667,0.0,0.023480,-1.0
7,30.942861,1.0,0.155763,inf
8,30.645283,0.0,-0.009617,-1.0
9,33.413870,1.0,0.090343,inf


In [15]:
df1 = df[['Peak_HI','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(30)

,Peak_HI,target,Peak_HI,target
0,31.224829,1.0,NaN,NaN
1,28.202562,1.0,-0.096791,0.0
2,29.479176,1.0,0.045266,0.0
3,28.942341,1.0,-0.018211,0.0
4,32.863735,1.0,0.135490,0.0
5,31.612535,1.0,-0.038072,0.0
6,31.910407,0.0,0.009423,-1.0
7,37.869746,1.0,0.186752,inf
8,35.806599,0.0,-0.054480,-1.0
9,42.549652,1.0,0.188319,inf


In [16]:
df1 = df[['Max_Humidity','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(50)

,Max_Humidity,target,Max_Humidity,target
0,79.34,1.0,NaN,NaN
1,76.92,1.0,-0.030502,0.0
2,73.11,1.0,-0.049532,0.0
3,64.66,1.0,-0.115579,0.0
4,74.42,1.0,0.150943,0.0
5,77.25,1.0,0.038027,0.0
6,80.83,0.0,0.046343,-1.0
7,92.66,1.0,0.146357,inf
8,87.42,0.0,-0.056551,-1.0
9,77.80,1.0,-0.110043,inf


In [17]:
## Days before current 

df1 = df[['Average_Current','MORNING_AVG_CURRENT_N_1','TARGET_N_1','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(50)

,Average_Current,MORNING_AVG_CURRENT_N_1,TARGET_N_1,target,Average_Current,MORNING_AVG_CURRENT_N_1,TARGET_N_1,target
0,4.352917,68.730303,0.0,1.0,NaN,NaN,NaN,NaN
1,4.416875,58.424242,0.0,1.0,0.014693,-0.149949,NaN,0.0
2,14.331250,36.430000,0.0,1.0,2.244658,-0.376457,NaN,0.0
3,27.535312,71.386364,0.0,1.0,0.921348,0.959549,NaN,0.0
4,0.845625,51.187273,0.0,1.0,-0.969289,-0.282954,NaN,0.0
5,9.230417,0.100000,0.0,1.0,9.915496,-0.998046,NaN,0.0
6,0.000000,29.520606,0.0,0.0,-1.000000,294.206061,NaN,-1.0
7,58.210104,37.409394,0.0,1.0,inf,0.267230,NaN,inf
8,60.773229,149.389091,0.0,0.0,0.044032,2.993358,NaN,-1.0
9,27.929583,138.279394,0.0,1.0,-0.540429,-0.074368,NaN,inf


In [15]:
df1 = df[['Average_Current','MORNING_AVG_CURRENT_N_2','TARGET_N_2','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(50)

,Average_Current,MORNING_AVG_CURRENT_N_2,TARGET_N_2,target,Average_Current,MORNING_AVG_CURRENT_N_2,TARGET_N_2,target
0,4.352917,19.471818,0.0,1.0,NaN,NaN,NaN,NaN
1,4.416875,34.783333,0.0,1.0,0.014693,0.786342,NaN,0.0
2,14.331250,45.837273,0.0,1.0,2.244658,0.317794,NaN,0.0
3,27.535312,32.867879,0.0,1.0,0.921348,-0.282944,NaN,0.0
4,0.845625,40.494848,0.0,1.0,-0.969289,0.232049,NaN,0.0
5,9.230417,43.980606,0.0,1.0,9.915496,0.086079,NaN,0.0
6,0.000000,9.588485,0.0,0.0,-1.000000,-0.781984,NaN,-1.0
7,58.210104,93.013939,0.0,1.0,inf,8.700588,NaN,inf
8,60.773229,85.711212,0.0,0.0,0.044032,-0.078512,NaN,-1.0
9,27.929583,52.059394,0.0,1.0,-0.540429,-0.392619,NaN,inf


In [18]:
df1 = df[['Avg_Heat_Index','AVERAGE_HEAT_INDEX_N_1','AVERAGE_HEAT_INDEX_N_2','TARGET_N_1','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(50)

,Avg_Heat_Index,AVERAGE_HEAT_INDEX_N_1,AVERAGE_HEAT_INDEX_N_2,TARGET_N_1,target,Avg_Heat_Index,AVERAGE_HEAT_INDEX_N_1,AVERAGE_HEAT_INDEX_N_2,TARGET_N_1,target
0,26.667515,26.667515,26.667515,0.0,1.0,NaN,NaN,NaN,NaN,NaN
1,24.975674,24.975674,24.975674,0.0,1.0,-0.063442,-0.063442,-0.063442,NaN,0.0
2,25.394108,25.394108,25.394108,0.0,1.0,0.016754,0.016754,0.016754,NaN,0.0
3,25.914466,25.914466,25.914466,0.0,1.0,0.020491,0.020491,0.020491,NaN,0.0
4,26.553766,26.553766,26.553766,0.0,1.0,0.024670,0.024670,0.024670,NaN,0.0
5,26.158478,26.158478,26.158478,0.0,1.0,-0.014886,-0.014886,-0.014886,NaN,0.0
6,26.772667,26.772667,26.772667,0.0,0.0,0.023480,0.023480,0.023480,NaN,-1.0
7,30.942861,30.942861,30.942861,0.0,1.0,0.155763,0.155763,0.155763,NaN,inf
8,30.645283,30.645283,30.645283,0.0,0.0,-0.009617,-0.009617,-0.009617,NaN,-1.0
9,33.413870,33.413870,33.413870,0.0,1.0,0.090343,0.090343,0.090343,NaN,inf


In [19]:
df1 = df[['Avg_humidity','AVERAGE_HUMUDITY_N_1','TARGET_N_1','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(50)

,Avg_humidity,AVERAGE_HUMUDITY_N_1,TARGET_N_1,target,Avg_humidity,AVERAGE_HUMUDITY_N_1,TARGET_N_1,target
0,60.078646,60.078646,0.0,1.0,NaN,NaN,NaN,NaN
1,69.590937,69.590937,0.0,1.0,0.158331,0.158331,NaN,0.0
2,62.813750,62.813750,0.0,1.0,-0.097386,-0.097386,NaN,0.0
3,50.201458,50.201458,0.0,1.0,-0.200789,-0.200789,NaN,0.0
4,64.328646,64.328646,0.0,1.0,0.281410,0.281410,NaN,0.0
5,63.816250,63.816250,0.0,1.0,-0.007965,-0.007965,NaN,0.0
6,66.228958,66.228958,0.0,0.0,0.037807,0.037807,NaN,-1.0
7,76.825417,76.825417,0.0,1.0,0.159997,0.159997,NaN,inf
8,64.606250,64.606250,0.0,0.0,-0.159051,-0.159051,NaN,-1.0
9,61.156250,61.156250,0.0,1.0,-0.053400,-0.053400,NaN,inf


In [20]:
df1 = df[['Avg_TEMP','AVERAGE_TEMP_N_1','TARGET_N_1','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(50)

,Avg_TEMP,AVERAGE_TEMP_N_1,TARGET_N_1,target,Avg_TEMP,AVERAGE_TEMP_N_1,TARGET_N_1,target
0,25.887917,25.887917,0.0,1.0,NaN,NaN,NaN,NaN
1,23.927083,23.927083,0.0,1.0,-0.075743,-0.075743,NaN,0.0
2,23.755521,23.755521,0.0,1.0,-0.007170,-0.007170,NaN,0.0
3,24.734896,24.734896,0.0,1.0,0.041227,0.041227,NaN,0.0
4,25.372917,25.372917,0.0,1.0,0.025794,0.025794,NaN,0.0
5,25.174583,25.174583,0.0,1.0,-0.007817,-0.007817,NaN,0.0
6,25.932083,25.932083,0.0,0.0,0.030090,0.030090,NaN,-1.0
7,27.889896,27.889896,0.0,1.0,0.075498,0.075498,NaN,inf
8,28.710729,28.710729,0.0,0.0,0.029431,0.029431,NaN,-1.0
9,30.336979,30.336979,0.0,1.0,0.056643,0.056643,NaN,inf


In [21]:
df1 = df[['Evening_Avg_Heat_Index','EVENING_AVG_TEMP_N_1','EVENING_AVG_TEMP_N_2','TARGET_N_1','target']]
a = df1.pct_change()
pd.concat([df1,a],axis=1).head(30)

,Evening_Avg_Heat_Index,EVENING_AVG_TEMP_N_1,EVENING_AVG_TEMP_N_2,TARGET_N_1,target,Evening_Avg_Heat_Index,EVENING_AVG_TEMP_N_1,EVENING_AVG_TEMP_N_2,TARGET_N_1,target
0,27.873791,26.834516,26.834516,0.0,1.0,NaN,NaN,NaN,NaN,NaN
1,25.922561,25.009032,25.009032,0.0,1.0,-0.070002,-0.068027,-0.068027,NaN,0.0
2,26.291401,25.344839,25.344839,0.0,1.0,0.014229,0.013427,0.013427,NaN,0.0
3,26.309882,25.615161,25.615161,0.0,1.0,0.000703,0.010666,0.010666,NaN,0.0
4,26.792381,25.911935,25.911935,0.0,1.0,0.018339,0.011586,0.011586,NaN,0.0
5,26.868280,26.086452,26.086452,0.0,1.0,0.002833,0.006735,0.006735,NaN,0.0
6,27.234939,26.378065,26.378065,0.0,0.0,0.013647,0.011179,0.011179,NaN,-1.0
7,33.409211,29.197097,29.197097,0.0,1.0,0.226704,0.106870,0.106870,NaN,inf
8,30.679703,28.889355,28.889355,0.0,0.0,-0.081699,-0.010540,-0.010540,NaN,-1.0
9,32.947427,30.095484,30.095484,0.0,1.0,0.073916,0.041750,0.041750,NaN,inf
